In [1]:
from ict_eeio.mrio import (
    get_all_industries,
    load_base,
    get_demand_categories,
    get_industry_output,
    get_monetary_flow,
    get_factor_input_per_Y_unit,
    get_leontief_flow,
    aggregate_sectors,
    get_total_final_demand,
)

from ict_eeio.eeio import (
    get_direct_impact_coefficient,
    get_cba_impact,
    get_pba_impact,
    get_total_direct_impact,
    get_all_satellites_stressors,
)
from ict_eeio.industries import ict_industries, energy_industries
from ict_eeio.satellites import (
    impact_stressors_used,
    impact_stressors_unused,
    satellites,
    impacts,
    clear_mineral_stressors_name,
)
from ict_eeio.sankey import plot_sankey
import pandas as pd
from collections import OrderedDict

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from ict_eeio.satellites import (
    mineral_stressors,
    mineral_stressors_used,
    mineral_stressors_unused,
)
from ict_eeio.utils import kg_to_Gt
from ict_eeio.mrio import simplify_sections
from ict_eeio.eeio import COE2_NAME
from ict_eeio.utils import kg_to_Mt

# Set seaborn theme for matplotlib
sns.set_theme()

/Users/thibaultsimon/Documents/projects/thesis/1_eeioa/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
year = 2020

exio_normal = load_base(year=str(year), classification="ixi")

exio_simplified = load_base(year=str(year), classification="ixi")
simplify_sections(exio_simplified)

# exio_agg = load_base(year=str(year), classification="ixi")
# aggregate_sectors(exio_agg, ict_industries, "ICT")

In [3]:
# For each ICT industry, we assess its upstream footprint
# If upstream is one of the other industries, we do not consider it
# If upstream is itself, it is direct

upstream_color = "red"
downstream_color = "blue"
direct_color = "black"

for exio in [exio_simplified, exio_normal]:
    labels = list(get_all_industries(exio) + get_demand_categories(exio))
    labels.append("Upstream")
    labels.append("Direct")
    labels.append("Downstream")
    labels.append("Total")
    labels.append("Direct up")

    sources = []
    targets = []
    values = []
    colors = []

    total_upstream = 0

    for industry in ict_industries:
        sum_y = exio.Y.sum(axis=1)
        S = exio.impacts.S[("global", industry)][COE2_NAME]

        direct = (
            exio.L[("global", industry)][("global", industry)]
            * sum_y[("global", industry)]
            * S
        )

        sources.append(labels.index("Direct"))
        targets.append(labels.index(industry))
        values.append(direct)
        colors.append(direct_color)

        sources.append(labels.index(industry))
        targets.append(labels.index("Total"))
        values.append(direct)
        colors.append(direct_color)

        upstream = 0
        for supplier in get_all_industries(exio):
            if supplier != industry:

                value = (
                    exio.L[("global", industry)][("global", supplier)]
                    * sum_y[("global", industry)]
                    * exio.impacts.S[("global", supplier)][COE2_NAME]
                )

                if supplier not in ict_industries:
                    sources.append(labels.index(supplier))
                    targets.append(labels.index("Upstream"))
                    values.append(value)
                    colors.append(upstream_color)
                else:
                    sources.append(labels.index("Direct up"))
                    targets.append(labels.index("Upstream"))
                    values.append(value)
                    colors.append(upstream_color)
                upstream += value

        sources.append(labels.index("Upstream"))
        targets.append(labels.index(industry))
        values.append(upstream)
        colors.append(upstream_color)

        sources.append(labels.index(industry))
        targets.append(labels.index("Total"))
        values.append(upstream)
        colors.append(upstream_color)

        print("---" + industry + "---")
        print("Upstream: " + str(upstream))
        print("Direct: " + str(direct))
        print("CBA: " + str(direct + upstream))
        print("CBA: " + str(get_cba_impact(exio, industry)) + " expected")
        print("Diff due to not double counting")
        print()
        total_upstream += upstream

    print("---Total---")
    print("Upstream: " + str(kg_to_Gt(total_upstream)))

    if exio == exio_simplified:
        plot_sankey(
            sources=sources,
            targets=targets,
            values=values,
            labels=labels,
            # colors=colors,
            regroup_small_nodes=True,
            regroup_threshold=55000000000 if exio == exio_simplified else 5500000000,
            width=10,
            height=5,
            title="",  # "ICT GHG emissions sources (2015)",
        ).write_image("figures/upstream_carbon.pdf")
        plot_sankey(
            sources=sources,
            targets=targets,
            values=values,
            labels=labels,
            # colors=colors,
            regroup_small_nodes=True,
            regroup_threshold=55000000000 if exio == exio_simplified else 5500000000,
            width=10,
            height=5,
            title="",  # "ICT GHG emissions sources (2015)",
        ).show()
    else:
        plot_sankey(
            sources=sources,
            targets=targets,
            values=values,
            labels=labels,
            # colors=colors,
            regroup_small_nodes=True,
            regroup_threshold=5500000000,
            width=10,
            height=5,
            title="",  # "ICT GHG emissions sources (2015)",
        ).show()  # .write_image("figures/greendays/impact_sources.svg")

    # print(upstream + direct, upstream, direct)
    # print(compute_udd_impact(exio, industry))

---Computer and related activities (72)---
Upstream: 303513792106.9861
Direct: 16488184072.751402
CBA: 320001976179.7375
CBA: 320001976179.7375 expected
Diff due to not double counting

---Manufacture of office machinery and computers (30)---
Upstream: 200806770849.6317
Direct: 8143413511.843322
CBA: 208950184361.47504
CBA: 208950184361.47507 expected
Diff due to not double counting

---Manufacture of radio, television and communication equipment and apparatus (32)---
Upstream: 402656081744.5714
Direct: 42301377571.13606
CBA: 444957459315.70746
CBA: 444957459315.7075 expected
Diff due to not double counting

---Post and telecommunications (64)---
Upstream: 217506100336.79962
Direct: 37232633935.748634
CBA: 254738734272.54825
CBA: 254738734272.54828 expected
Diff due to not double counting

---Total---
Upstream: 1.1244827450379888
Filling colors
{29: 104165609091.47943, 15: 640003952359.475, 31: 1228648354129.4683, 0: 116537028366.5463, 28: 2248965490075.9785, 1: 407638987.5457492, 2: 1

---Computer and related activities (72)---
Upstream: 302015113787.28436
Direct: 16508150040.680891
CBA: 318523263827.9653
CBA: 318523263827.96515 expected
Diff due to not double counting

---Manufacture of office machinery and computers (30)---
Upstream: 221937635416.69507
Direct: 8148147457.9809675
CBA: 230085782874.67603
CBA: 230085782874.67593 expected
Diff due to not double counting

---Manufacture of radio, television and communication equipment and apparatus (32)---
Upstream: 429113697652.9008
Direct: 42390898553.52815
CBA: 471504596206.42896
CBA: 471504596206.4289 expected
Diff due to not double counting

---Post and telecommunications (64)---
Upstream: 240879548675.1554
Direct: 37265899222.52882
CBA: 278145447897.6842
CBA: 278145447897.6842 expected
Diff due to not double counting

---Total---
Upstream: 1.1939459955320357
Filling colors
{171: 104313095274.71883, 132: 637046527655.9304, 173: 1298259090806.7544, 0: 10113702136.307693, 170: 2387891991064.074, 1: 2826425283.199013,